In [1]:
%load_ext autoreload
%autoreload 2


In [35]:
from psql_helpers import read_table_async
from psql_models import UserMeeting,User,Meeting,UserToken
from sqlalchemy import select

In [36]:
await read_table_async(UserToken)

,token,user_id,user_name,created_at,last_used_at
0,3ae04e20124d40babc5107e658c666b6,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,Dmitriy Grankin,2024-10-29 15:22:09.758227+00:00,2024-10-30 07:10:25.609637+00:00
1,5faa86ba0dc74b14a78c60c8aa20a014,8b767b85-44fb-4c20-97df-7aa172242273,Olga Nemirovskaya,2024-10-29 15:55:22.473248+00:00,2024-10-30 07:11:19.311435+00:00


In [3]:
await read_table_async(UserMeeting)

,id,meeting_id,user_id,access_level,created_at,created_by,is_owner
0,1,7343734f-7311-4844-aa5e-86f1f93595c5,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,owner,2024-10-29 15:22:23.365912+00:00,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,True
1,2,50d5d659-3a57-42f2-8f32-31c7145ebe91,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,owner,2024-10-29 15:22:59.524789+00:00,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,True
2,3,5824bf7d-b528-429f-90fe-7048fa932539,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,owner,2024-10-29 15:23:24.426869+00:00,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,True
3,4,43fb67c6-9a6b-4fe2-88e1-ced34d0a47e3,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,owner,2024-10-29 15:23:30.214269+00:00,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,True
4,5,9b4c6346-9510-467e-91f6-3e6fd3bc42c7,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,owner,2024-10-29 15:23:38.332628+00:00,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,True
...,...,...,...,...,...,...,...
214,215,735503e6-d4d6-4865-bf59-2ebc4217852a,8b767b85-44fb-4c20-97df-7aa172242273,owner,2024-10-29 18:03:30.801286+00:00,8b767b85-44fb-4c20-97df-7aa172242273,True
215,216,3d65802a-7a43-447c-b21b-ab2ee7390970,8b767b85-44fb-4c20-97df-7aa172242273,owner,2024-10-29 18:03:32.462355+00:00,8b767b85-44fb-4c20-97df-7aa172242273,True
216,217,117f4cb0-1641-4647-b534-ee3176763569,8b767b85-44fb-4c20-97df-7aa172242273,owner,2024-10-29 18:03:38.317768+00:00,8b767b85-44fb-4c20-97df-7aa172242273,True
217,218,cb8565be-75c3-4a64-93ae-264530ef39c3,8b767b85-44fb-4c20-97df-7aa172242273,owner,2024-10-30 06:31:01.982226+00:00,8b767b85-44fb-4c20-97df-7aa172242273,True


In [12]:
query = select(
    UserMeeting.meeting_id.label('meeting_id'),
    UserMeeting.access_level.label('access_level'),
    UserMeeting.is_owner.label('is_owner'),
    User.username.label('username'),
).join(User, UserMeeting.user_id == User.id)
result = await read_table_async(query)

In [14]:
result['username'].unique()

array(['Dmitriy Grankin', 'Olga Nemirovskaya'], dtype=object)

In [16]:
query = select(
    Meeting)

In [42]:
query = select(
    Meeting.meeting_id,
    Meeting.timestamp,
    Meeting.transcript
).join(
    UserMeeting, Meeting.meeting_id == UserMeeting.meeting_id
).join(
    User, UserMeeting.user_id == User.id
).where(
    User.username == 'Olga Nemirovskaya'
)


In [43]:
meeting_df = await read_table_async(query)

In [45]:
meeting_df.sort_values('timestamp', ascending=False)

,meeting_id,timestamp,transcript
0,7b67caaa-fec0-440c-a4a1-b09064a00e81,2024-10-29 11:16:20.144000,"[{'speaker': 'TBD', 'speaker_id': 'TBD', 'cont..."
25,e51f426f-13b5-4f8a-8846-6a5149bcb47e,2024-10-22 11:03:09.290000,"[{'speaker': 'Дмитрий Кочетов', 'speaker_id': ..."
1,0f1e7e9e-eec7-4815-8406-89bd38c5fd16,2024-10-15 11:04:42.820000,"[{'speaker': 'Виктория Сёмка', 'speaker_id': '..."
2,831b8187-57ab-4662-8cfe-9abcac2b9bb0,2024-10-14 13:01:29.304000,"[{'speaker': 'Olga Nemirovskaya', 'speaker_id'..."
3,ba71dc8a-5088-4efe-abb8-9ab092488d15,2024-10-12 08:00:32.084000,"[{'speaker': 'Lola Aleksandrova', 'speaker_id'..."
...,...,...,...
88,8cf049d5-5b42-45fa-9ae6-e8130aef3435,2024-08-21 15:04:09.424818,"[{'speaker': 'click to name speaker', 'speaker..."
91,117f4cb0-1641-4647-b534-ee3176763569,2024-08-21 09:50:25.384507,"[{'speaker': 'click to name speaker', 'speaker..."
87,8e8b8dbe-7238-4d22-a3c1-2d8719a8728b,2024-08-21 09:36:15.303366,"[{'speaker': 'Spectral Spider', 'speaker_id': ..."
90,3d65802a-7a43-447c-b21b-ab2ee7390970,2024-08-20 19:19:03.123598,"[{'speaker': 'Tempest Dragon', 'speaker_id': '..."


In [40]:
meeting_df.sort_values('timestamp', ascending=False)

KeyError: 'timestamp'

In [26]:
import pandas as pd

# Assuming meeting_df['transcript'].tolist() gives you the list of strings
transcripts = meeting_df['transcript'].tolist()

# Convert to DataFrame
df = pd.DataFrame(eval(transcripts[0]))  # Assuming the first item is representative of the structure

# If you need to process all transcripts:
all_transcripts = []
for transcript in transcripts:
    if transcript:  # Check if not None/empty
        all_transcripts.extend(eval(transcript))
        
df = pd.DataFrame(all_transcripts)

In [29]:
speaker_counts = df.groupby('speaker').size().reset_index(name='count').sort_values(by='count', ascending=False)

In [34]:
speaker_counts.head(10)

,speaker,count
15,Dmitriy Grankin,8366
43,Olga Nemirovskaya,6745
58,TBD,3497
79,Елизавета Иванова,2569
52,Sergey Ryabenko,1795
17,Dmitry Grankin,1692
78,Егор Мараев,1330
23,Ilia Semukhin,586
75,Дмитрий Кочетов,571
85,Саша Алексеев,505


In [46]:
df[df['speaker'] == 'Olga Nemirovskaya'].sort_values('timestamp')

,speaker,speaker_id,content,html_content,html_content_short,keywords,timestamp
37644,Olga Nemirovskaya,TBD,"Я скачала через Chrome новый, потому что у ме...","Я скачала новый <b>Chrome</b>, потому что у ме...","Chrome, неопубликованный",[],2024-08-26T09:46:22.334000Z
37645,Olga Nemirovskaya,TBD,"Я скачала, и тот удалила.","Я скачала, и тот удалил.",,[],2024-08-26T09:46:27.044000Z
37646,Olga Nemirovskaya,TBD,"Так, что-то у нас тут какое-то прикольное кол...","Так, у нас появилось какое-то <b>прикольное ко...",прикольное колесико,[],2024-08-26T09:46:32.684000Z
37647,Olga Nemirovskaya,TBD,Оно меня отправляет на Success...,Оно меня отправляет на <b>Success</b>...,Success,[],2024-08-26T09:46:38.144000Z
37649,Olga Nemirovskaya,TBD,"Да, так, ну, короче, я не заметила этот перек...","Да, так, короче, я не заметила этот <b>переклю...",переключатель,[],2024-08-26T09:47:12.200000Z
...,...,...,...,...,...,...,...
10469,Olga Nemirovskaya,TBD,Сейчас я пойду переваривать информацию.,Сейчас я пойду <b>переваривать</b> информацию.,переваривать,[],2024-10-14T13:29:55.108000Z
10470,Olga Nemirovskaya,TBD,"Да, слушай, наверное, я бы вернулась.","Да, слушай, наверное, я бы <b>вернулась</b>.",вернулась,[],2024-10-14T13:29:57.468000Z
10471,Olga Nemirovskaya,TBD,"Может быть, через недельку бы осмыслила, что-...","Может быть, через <b>недельку</b> я осмыслила ...",недельку,[],2024-10-14T13:30:05.524000Z
10472,Olga Nemirovskaya,TBD,"Да, конечно.","Да, конечно.",,[],2024-10-14T13:30:06.944000Z
